## Part 2
#### Load training file and create dictionary containing the occurrences of each annotation in the training set

In [2]:
def load_original_train():
    with open("train", encoding="utf-8") as file:
        train_list = file.readlines()
        train_list = [x.strip() for x in train_list]
        return train_list
    
train_list = load_original_train()

In [3]:
annotationDict = {"O": 0, "I-positive": 0, "B-positive": 0, "I-negative":0, "B-negative": 0, "I-neutral":0, "B-neutral":0}

for i in range(len(train_list)):
    annotated_word = train_list[i]
    each_word = annotated_word.split(" ")
    if (len(each_word) == 2):
        if each_word[1] == "O":
            annotationDict["O"] += 1
        elif each_word[1] == "I-positive":
            annotationDict["I-positive"] += 1
        elif each_word[1] == "B-positive":
            annotationDict["B-positive"] += 1
        elif each_word[1] == "I-negative":
            annotationDict["I-negative"] += 1
        elif each_word[1] == "B-negative":
            annotationDict["B-negative"] += 1
        elif each_word[1] == "I-neutral":
            annotationDict["I-neutral"] += 1
        elif each_word[1] == "B-neutral":
            annotationDict["B-neutral"] += 1

print(annotationDict)

{'O': 24242, 'I-positive': 607, 'B-positive': 1208, 'I-negative': 133, 'B-negative': 382, 'I-neutral': 23, 'B-neutral': 65}


#### (a) Write a function that estimates the emission parameters from the training set using MLE

In [4]:
def estimate_emission(word, tag):
    countTop = 0
    countBottom = annotationDict[tag]
    for i in range(len(train_list)):
        annotated_word = train_list[i]
        each_word = annotated_word.split(" ")
        if len(each_word) == 2:
            if (each_word[0] == word and each_word[1] == tag):
                countTop += 1

#         print(countTop)
#         print(countBottom)
    return countTop/countBottom

estimate_emission("service", "O")

0.0004950086626515964

#### (b) Replace the words that appear less than k times in the training set with a special token #UNK# before training. This leads to a “modified training set”.

In [5]:
import datetime

def modify_trainingset(k):
        wordCountDict = {}
        for i in range(len(train_list)):
            annotated_word = train_list[i]
            each_word = annotated_word.split(" ")
            if each_word[0] in wordCountDict:
                wordCountDict[each_word[0]] += 1
            else:
                wordCountDict[each_word[0]] = 1
        
       
        wordToBeReplacedDict = {}
        wordToBeReplacedList = []
        for key in wordCountDict:
            if wordCountDict[key] < k:
                wordToBeReplacedList.append(key)

        start = datetime.datetime.now()
        for i in range(len(train_list)): 
            annotated_word = train_list[i]
            each_word = annotated_word.split(" ")
            if each_word[0] in wordToBeReplacedList:
                train_list[i] = "#UNK# " + each_word[1]


        modifiedTotalTrainSet = "\n".join(train_list)

        f = open("modified_train","w+")
        f.write(modifiedTotalTrainSet)
                            
        end = datetime.datetime.now()
        elapsed = end - start
        print(elapsed)

modify_trainingset(3)

0:00:01.208553


#### Loads the modified training set into a list, and count the occurences of each word in the list and store it into a dictionary called modifiedWordDict

In [6]:
def load_modified_train_file():
    with open("modified_train", encoding="utf-8") as file:
        modified_train_list = file.readlines()
        modified_train_list = [x.strip() for x in modified_train_list]
        return modified_train_list
        
modified_train_list = load_modified_train_file()

In [7]:
def get_word_dict():
    modifiedWordDict = {}        
    for i in range(len(modified_train_list)):
        annotated_word = modified_train_list[i]
        each_word = annotated_word.split(" ")
        if each_word[0] in modifiedWordDict:
            modifiedWordDict[each_word[0]] += 1
        else:
            modifiedWordDict[each_word[0]] = 1

    return modifiedWordDict
    
modifiedWordDict = get_word_dict()

In [8]:
def count_word_tag_pair_dict():
    word_tag_pair_dict= {}
    for word_tag_pair in modified_train_list:
        if word_tag_pair_dict.get(word_tag_pair) == None:
            word_tag_pair_dict[word_tag_pair] = 1
        else:
            word_tag_pair_dict[word_tag_pair] += 1
    return word_tag_pair_dict

    
wordTagPairDict = count_word_tag_pair_dict()

#### Implement a fixed emission function

In [9]:
def estimate_emission_fix(word, tag):
        countTop = 0
        countBottom = annotationDict[tag]
        if word in modifiedWordDict:
            # do nothing
            word = word
            
        else:
            word = "#UNK#"

        for i in range(len(modified_train_list)):
            annotated_word = modified_train_list[i]
            each_word = annotated_word.split(" ")
            if (each_word[0] == word and each_word[1] == tag):
                countTop += 1
        return countTop/countBottom

estimate_emission_fix("I", "O")

0.019511591452850426

#### Loop through the words once and store the emission value of each word tag pair into a giant dictionary called giantEmissionDict

In [64]:
def store_estimate_emission_fix():
    giantEmissionDict = {}
    for word_tag_pair in modified_train_list:
        if word_tag_pair != "":
            word_tag_pair_list = word_tag_pair.split(" ")
            countTop = wordTagPairDict[word_tag_pair]
            countBottom = modifiedWordDict[word_tag_pair_list[0]]
            emission = countTop/countBottom
            if giantEmissionDict.get(word_tag_pair) == None:
                giantEmissionDict[word_tag_pair] = countTop/countBottom
    return giantEmissionDict
    
giantEmissionDict = store_estimate_emission_fix()
print(giantEmissionDict)

{'We O': 1.0, 'were O': 1.0, 'then O': 1.0, '#UNK# O': 0.8190255220417634, 'for O': 0.9928825622775801, 'their O': 1.0, 'most O': 1.0, 'expensive O': 1.0, 'sake O': 0.4, '( O': 0.9302325581395349, '$ O': 0.9583333333333334, '20 O': 1.0, '+ O': 0.8, 'per O': 1.0, 'serving O': 1.0, ') O': 0.9325842696629213, 'when O': 1.0, 'we O': 1.0, 'in O': 0.9846153846153847, 'fact O': 1.0, 'a O': 1.0, 'of O': 0.9426751592356688, 'less O': 1.0, 'than O': 1.0, 'half O': 0.75, 'that O': 1.0, 'price O': 0.9333333333333333, '. O': 1.0, '#UNK# B-neutral': 0.0031902552204176333, 'is O': 1.0, 'nothing O': 1.0, 'special O': 0.5384615384615384, 'Great O': 1.0, '#UNK# B-positive': 0.0849767981438515, 'and O': 0.9732824427480916, 'worth O': 1.0, 'every O': 1.0, 'bit O': 1.0, 'delicious O': 1.0, 'bagels B-positive': 0.625, ', O': 0.9947257383966245, 'especially O': 1.0, 'right O': 1.0, 'out O': 1.0, 'the O': 0.9936823104693141, 'Go O': 0.8181818181818182, 'here O': 1.0, 'Highly O': 1.0, 'recommended O': 1.0, '! 

#### (c) Implement a simple sentiment analysis system that produces the tag

In [92]:
def sentiment_analysis(word):
    all_tags = ["O", "B-positive", "I-positive", "B-negative", "I-negative", "B-neutral", "I-neutral"]
    scoreDict = {}
    # if word never appeared before, let it be #UNK#
    if word in modifiedWordDict:
        word = word
    else:
        word = "#UNK#"
    for tag in all_tags:
        possible_word_tag = word + " " + tag
        if giantEmissionDict.get(possible_word_tag) == None:
            scoreDict[tag] = 0
        else:
            scoreDict[tag] = giantEmissionDict[possible_word_tag]
            
    maximumScore = max(scoreDict.values())
    return max(scoreDict, key=scoreDict.get)

   
sentiment_analysis("ambience")

'B-positive'

#### Loads dev.in file and store as dev_in_list

In [12]:
def load_files():
    with open("dev.in", encoding="utf-8") as file:
        dev_in_list = file.readlines()
        dev_in_list = [x.strip() for x in dev_in_list]
        return dev_in_list
        
dev_in_list = load_files()

In [15]:
import datetime
def predict_sentiment():
    wordDict = {}
    f = open("dev.p2.out","w+")
    giantStringToBeWritten = ""
    start = datetime.datetime.now()
    for i in range(len(dev_in_list)):
        each_word = dev_in_list[i]
        if each_word in wordDict:
            tag = wordDict[each_word]
            giantStringToBeWritten += each_word +  " " + tag + "\n"

        else:
            if each_word != "":
                tag = sentiment_analysis(each_word)
                wordDict[each_word] = tag 
                giantStringToBeWritten += each_word +  " " + tag + "\n"

            else:
                giantStringToBeWritten += " \n"


    f.write(giantStringToBeWritten)
    end = datetime.datetime.now()
    elapsed = end - start
    print(elapsed)

predict_sentiment()

0:00:00.015362


### Part 3 
#### Function to estimate Transition Parameters using MLE

#### Creating a giant_each_tweet_tag that contains only the states of each sentence

In [33]:
def separate_tweets():
    countStart = 1
    # counting starts
    giant_each_tweet = []
    each_tweet = []
    totalStateNumber = {}
    for i in range(len(modified_train_list)):
        annotated_word = modified_train_list[i]
        each_word = annotated_word.split(" ")
        if each_word == ['']:
            countStart += 1
            giant_each_tweet.append(each_tweet)
            each_tweet = []

        else:
            each_tweet.append(each_word[1])
    for i in range(len(giant_each_tweet)):
        giant_each_tweet[i].insert(0, 'START')
        giant_each_tweet[i].append('STOP')
    
    for each_tweet in giant_each_tweet:
        for tag in each_tweet:
            if totalStateNumber.get(tag) == None:
                totalStateNumber[tag] = 1
            else:
                totalStateNumber[tag] += 1

    return giant_each_tweet, countStart, totalStateNumber
giantTaggedEachTweet, countStart, totalStateNumber = separate_tweets()
print(totalStateNumber)

{'START': 1872, 'O': 24232, 'STOP': 1872, 'B-neutral': 65, 'B-positive': 1208, 'B-negative': 381, 'I-negative': 133, 'I-positive': 607, 'I-neutral': 23}


In [34]:
def store_estimate_transition():
    giantTransitionDict = {}
    finalGiantTransitionDict = {}
    for taggedEachTweet in giantTaggedEachTweet:
        for i in range(len(taggedEachTweet) - 1):
            if giantTransitionDict.get(taggedEachTweet[i]+" "+taggedEachTweet[i+1]) == None:
                giantTransitionDict[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] = 1
            else:
                giantTransitionDict[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] += 1
    for eachTransition in giantTransitionDict:
        eachTransitionList = eachTransition.split(" ")
        startState = eachTransitionList[0]
        countTop = giantTransitionDict[eachTransition]
        countBottom = totalStateNumber[startState]
        transitionParam = countTop/countBottom
        finalGiantTransitionDict[eachTransition] = transitionParam
        
    return finalGiantTransitionDict
    
giantTransitionDict = store_estimate_transition()
print(giantTransitionDict)

{'START O': 0.9401709401709402, 'O O': 0.8601436117530538, 'O STOP': 0.07622152525586003, 'START B-neutral': 0.005341880341880342, 'B-neutral O': 0.7846153846153846, 'O B-positive': 0.04646748101683724, 'B-positive O': 0.6887417218543046, 'O B-negative': 0.014897655992076594, 'B-negative O': 0.7821522309711286, 'B-negative I-negative': 0.2099737532808399, 'I-negative I-negative': 0.39849624060150374, 'I-negative O': 0.6015037593984962, 'O B-neutral': 0.002269725982172334, 'B-positive I-positive': 0.2980132450331126, 'I-positive O': 0.5848434925864909, 'I-positive I-positive': 0.40691927512355847, 'START B-positive': 0.0438034188034188, 'B-neutral I-neutral': 0.2, 'I-neutral I-neutral': 0.43478260869565216, 'I-neutral O': 0.5652173913043478, 'B-negative STOP': 0.007874015748031496, 'START B-negative': 0.010683760683760684, 'I-positive STOP': 0.008237232289950576, 'B-positive STOP': 0.013245033112582781, 'B-neutral STOP': 0.015384615384615385}


In [141]:

def extract_path(piLayer, wordList):
    maximum = max(piLayer, key=piLayer.get)
    best_path = piLayer[maximum][0]
    best_path_list = best_path.split(',')
    best_path_list.remove('START')
    best_path_list.remove('STOP')
    print(best_path_list)
    print(wordList)
    giantStringToBeWrittenToFile = ""
    for i in range(len(best_path_list)):
        if (wordList[i] == ''):
            giantStringToBeWrittenToFile += " \n"
        else:
            giantStringToBeWrittenToFile += wordList[i] + " " + best_path_list[i] + "\n"

    return giantStringToBeWrittenToFile

        
wordList = ['When', 'I', 'called', 'this', 'morning', ',', 'I', "didn't", 'think', 'I', 'would', 'be', 'able', 'to', 'get', 'in', 'at', '12', ',', 'but', 'I', 'was', 'able', 'to', 'get', 'in', ',', 'along', 'with', 'four', 'other', 'guests', '.', '']
lastPiLayer = {'O': ['START,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,STOP', 1.639577736548346e-87]}
extract_path(lastPiLayer,wordList)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['When', 'I', 'called', 'this', 'morning', ',', 'I', "didn't", 'think', 'I', 'would', 'be', 'able', 'to', 'get', 'in', 'at', '12', ',', 'but', 'I', 'was', 'able', 'to', 'get', 'in', ',', 'along', 'with', 'four', 'other', 'guests', '.', '']


"When O\nI O\ncalled O\nthis O\nmorning O\n, O\nI O\ndidn't O\nthink O\nI O\nwould O\nbe O\nable O\nto O\nget O\nin O\nat O\n12 O\n, O\nbut O\nI O\nwas O\nable O\nto O\nget O\nin O\n, O\nalong O\nwith O\nfour O\nother O\nguests O\n. O\n \n"

In [142]:
def viterbi():
    all_tags=["O","B-negative","B-positive","B-neutral","I-negative","I-positive","I-neutral"]
    beginning= True
    piList = []
    wordList = []
    giantStringToBeWritten = ""
    for word in dev_in_list:
        if word in modifiedWordDict:
            word = word
        else:
            word = "#UNK#"
        
        piLayer = {}
        if beginning:
            piList = []
            wordList = []
            for tag in all_tags:
                # if it does not exist in the dict, that means it will be 0, and we dont have to consider those cases
                if giantTransitionDict.get("START" + " " + tag) != None and giantEmissionDict.get(word + " " + tag) != None:
                    pi = giantTransitionDict["START" + " " + tag]*giantEmissionDict[word + " " + tag]
                    piLayer[tag] = ["START," + tag, pi]
            piList.append(piLayer)
            wordList.append(word)
            beginning = False
        
        else:
            previousLayer = piList[len(piList)-1]
            if word == "":
                tempLayer = {} 
                for previous_tag in previousLayer:
                    if giantTransitionDict.get(previous_tag + " " + "STOP") != None:
                        pi = previousLayer[previous_tag][1]*giantTransitionDict[previous_tag + " " + "STOP"]
                        tempLayer[previous_tag] = pi

                if len(tempLayer) != 0:
                    maxPiTag = max(tempLayer, key=tempLayer.get)
                    maximumPiScore = max(tempLayer.values())
                    tempLayer[maxPiTag] = [previousLayer[previous_tag][0]+","+maxPiTag + ",STOP",maximumPiScore]
                    wordList.append("")
                    singleStringToBeWritten = extract_path(tempLayer, wordList)
                    giantStringToBeWritten += singleStringToBeWritten
                beginning = True

            else:    
                piLayer = {}
                for previous_tag in previousLayer:
                    tempLayer = {}
                    for tag in all_tags:
                        if giantTransitionDict.get(previous_tag + " " + tag) != None and giantEmissionDict.get(word + " " + tag) != None:
                            pi = previousLayer[previous_tag][1]*giantTransitionDict[previous_tag + " " + tag]*giantEmissionDict[word + " " + tag]
                            tempLayer[tag] = pi
                    
                    if len(tempLayer) != 0:
                        maxPiTag = max(tempLayer, key=tempLayer.get)
                        maximumPiScore = max(tempLayer.values())
                        piLayer[maxPiTag] = [previousLayer[previous_tag][0]+","+maxPiTag, maximumPiScore]
                wordList.append(word)
                piList.append(piLayer)
#                 piLayer = {}
#                 for previous_tag in previousLayer:
#                     tempLayer = {}
#                     for tag in all_tags:
#                         if giantTransitionDict.get(previous_tag + " " + tag) != None and giantEmissionDict.get(word + " " + tag) != None:
#                             pi = previousLayer[previous_tag][1]*giantTransitionDict[previous_tag + " " + tag]*giantEmissionDict[word + " " + tag]
#                             tempLayer[tag] = pi
                    
#                     if len(tempLayer) != 0:
#                         maxPiTag = max(tempLayer, key=tempLayer.get)
#                         maximumPiScore = max(tempLayer.values())
#                         piLayer[maxPiTag] = [previousLayer[previous_tag][0]+","+maxPiTag, maximumPiScore]
#                 wordList.append(word)
#                 piList.append(piLayer)

    f = open("dev.p3.out","w+")
    f.write(giantStringToBeWritten)
   
    
viterbi()

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['When', 'I', 'called', 'this', '#UNK#', ',', 'I', "didn't", 'think', 'I', 'would', 'be', 'able', 'to', 'get', 'in', 'at', '#UNK#', ',', 'but', 'I', 'was', 'able', 'to', 'get', 'in', ',', '#UNK#', 'with', 'four', 'other', 'guests', '.', '']
['B-neutral', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['The', 'tuna', 'and', '#UNK#', 'potatoes', 'are', 'excellent', '.', '']
['O', 'O', 'O', 'O', 'O', 'O', 'O']
['My', '#UNK#', 'pizza', 'joint', 'in', 'Seattle', '']
['B-neutral', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['Food', '#UNK#', ',', 'its', 'ok', 'but', 'a', 'bit', 'pricey', 'for', 'what', 'you', 'get', 'considering', 'the', 'restaurant', '#UNK#', 'a', '#UNK#', 'place', '.', '']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['–', '#UNK#', 'away', 'over'